In [76]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda,UpSampling2D, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam,RMSprop
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                        import Path
from PIL                            import Image

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
from skimage                        import img_as_ubyte
from skimage                        import io
import shutil
tf.keras.backend.clear_session()

#### <font color="red">1. Load dataset</font>

Load NumPy arrays with tf.data.Dataset. Passing the two arrays as a tuple into tf.data.Dataset.from_tensor_slices to create a tf.data.Dataset.


In [95]:
specific_directory = '/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked'

path_train ='/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/train'
path_test ='/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/test'
path_val= '/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/valid'

nbatch = 10

def dataset_array(flow_path,geo_path):
    def load_array(flow,geo):
        x = np.load(flow)
        y = np.load(geo)
        return x,y

    x, y = tf.numpy_function(load_array, [flow_path, geo_path], [tf.float64, tf.float64])
    x.set_shape([x.shape[0],x.shape[1],x.shape[2]])
    y.set_shape([y.shape[0],y.shape[1],y.shape[2]])
    return x,y

def create_dataset(flow_path,geo_path,batch_size = nbatch):
    x_files = sorted(glob.glob(os.path.join(flow_path, "*.npy")))
    y_files = sorted(glob.glob(os.path.join(geo_path, "*.npy")))
    dataset = tf.data.Dataset.from_tensor_slices((x_files, y_files))
    dataset = dataset.map(dataset_array, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    return dataset

    
    

In [96]:
geo_path_train='/home/ppgi/Trabajo/predicting-flow-patterns/G_Masked/train'
p_path_train='/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/train'

geo_path_valid='/home/ppgi/Trabajo/predicting-flow-patterns/G_Masked/valid'
p_path_valid='/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/valid'

geo_path_test='/home/ppgi/Trabajo/predicting-flow-patterns/G_Masked/test'
p_path_test='/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/test'

train_ds =create_dataset(geo_path_train,p_path_train)
test_ds  =create_dataset(geo_path_test,p_path_test)
valid_ds =create_dataset(geo_path_valid,p_path_valid)

#### <font color="red">3. Hiperparameters</font>

In [ ]:
num_epochs = 50
patience=25  # How long to wait after last time validation loss improved
LR=0.001
# Model name
model_name="U-Net"
save_in='/home/ppgi/Trabajo/predicting-flow-patterns'

# image dimensions
img_width             =  128   # 739   G:737
img_height            =  32   # 185
channel               =  1

number_of_filters = [64, 128, 256, 512, 1024]

type_padding = 'same'
f_activation = 'relu'
f_activation_last='relu'
optimizer = RMSprop(learning_rate=LR)
#optimizer = Adam(learning_rate=LR)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',  min_delta=1e-2, patience=patience, restore_best_weights=True,verbose=1)
save = save_in + '/Results_weights/Best_weights.weights.h5'
checkpoint_cb=tf.keras.callbacks.ModelCheckpoint(save,save_weights_only=True)


#### <font color="red">4. Architecture</font>

In [ ]:
def conv_block_batchnorm(filters,x):
    conv = Conv2D(filters, (3, 3), padding=type_padding)(x)
    conv= BatchNormalization()(conv)
    conv = Activation(f_activation)(conv)
    conv = Conv2D(filters, (3, 3), padding=type_padding)(conv)
    conv= BatchNormalization()(conv)
    conv = Activation(f_activation)(conv)
    return conv
    
def conv_block(filters,x):
    conv = Conv2D(filters, (3, 3), activation=f_activation, padding=type_padding)(x)
    conv = Conv2D(filters, (3, 3), activation=f_activation, padding=type_padding)(conv)
    return conv    

def encoder(x,filters):
    conv = conv_block_batchnorm(filters,x)
    downsample = MaxPooling2D((2,2))(conv)
    return conv,downsample

def decoder(x1,x2,filters,transpose=None):
    if transpose != None:
        conv_up = Conv2DTranspose(filters,(2,2),strides=(2, 2),padding=type_padding)(x1)
    else:
        conv_up = UpSampling2D((2, 2))(x1)
   
    concat=concatenate([conv_up,x2],axis = 3)
    up = conv_block(filters, concat)
    return up


In [ ]:
# number_of_filters = [64, 128, 256, 512, 1024]
def make_model():
    
    image_input = Input((img_height, img_width, channel))
    
    conv1,down_block1 = encoder (image_input, number_of_filters[0])
    conv2,down_block2 = encoder (down_block1 , number_of_filters[1])
    conv3,down_block3 = encoder (down_block2 , number_of_filters[2])
    conv4,down_block4 = encoder (down_block3 , number_of_filters[3])

    return model  